In [1]:
import os

In [2]:
%pwd

'/mnt/e/cv_mlops/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/mnt/e/cv_mlops'

In [26]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [27]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [28]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH, 
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir, 
            source_URL = config.source_URL, 
            local_data_file = config.local_data_file, 
            unzip_dir = config.unzip_dir)

        return data_ingestion_config

In [29]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [34]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL, 
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downlaod! with followign info: \n{headers}")
        else:
            logger.info(f"File already exist of size:{get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        '''
            zip_file_path: str
            Extracts teh zip file in to the data directory 
            Function return None
        '''
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [35]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config= data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-08-24 11:42:47,041: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-08-24 11:42:47,051: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-24 11:42:47,059: INFO: common: created directory at: artifacts]
[2023-08-24 11:42:47,074: INFO: common: created directory at: artifacts/data_ingestion]
[2023-08-24 11:42:47,089: INFO: 4013225635: File already exist of size:~ 11345 KB]
